In [21]:
file_name_save = "IPB-20210524.xlsx" #ダウンロードされるファイル名

In [22]:
import requests,json,sys,os, gspread, time, re, openpyxl
import numpy as np
if 'google.colab' in str(get_ipython()):
    %pip install python-docx
    from google.colab import files,auth
    from oauth2client.client import GoogleCredentials
    outputdirectory = ''
else:
    outputdirectory = '../docx-researchmap-outputs/' #ローカルで実行する場合は保存ファイルのディレクトリを適当に指定
    os.makedirs(outputdirectory,exist_ok=True)
import pandas as pd
file_name=outputdirectory+file_name_save

In [23]:
#スプレッドシートをダウンロード
sheeturl='https://docs.google.com/spreadsheets/d/1wce1XHSFGSBttupnSIqe_5abtijBb_hBYM2bfaV9Jn4/edit#gid=0' #新学術全員入りのリストのスプレッドシート
sheeturl_csv=re.match("https://docs.google.com/spreadsheets/d/.+/",sheeturl).group(0)+"export?format=csv"
name_data=pd.read_csv(sheeturl_csv)
allnames=(name_data["Surname"]+' '+name_data["First name"]).to_list()
allSurname=name_data["Surname"].to_list()
allnamesJP=(name_data["苗字"]+" "+name_data["名"]).to_list()
allgroupnames=name_data["班"].to_list()
allgroupnum=name_data["番号"].to_list()
allmembers=name_data["researchmapID"].to_list()
allDB=name_data["代表分担協力"].values
allkeikaku=[b for a,b in zip(allgroupnames,allnamesJP) if a in ['A','B','C']]
allkeikakuPIs=[b for a,b,c in zip(allgroupnames,allnamesJP,allDB) if (a in ['A','B','C']) & (c =='D')]
allDaihyoBuntan=list(allDB)
grant_numbers=name_data["grantID"].to_list()
allmindate=name_data["Start date"].to_list()
allmaxdate=name_data["End date"].to_list()

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45
0,2021/05/12 17:24:27,kyogo.kawaguchi@riken.jp,川口 喬吾,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,IMBA・RIVRON lab、Broad Insititute・Singh lab、Yal...,NaN,NaN,NaN,アクティブマター勉強会（岡田研、竹内研、川口研）、A班内共同会議（岡田研、佐々研、川口研）,NaN,NaN,理研BDRのHP改訂、ダイバーシティ・女性関連のページ作成https://www.bdr.r...,IPBセミナーにて、海外在住の若手研究者の発表の機会を作った（花井、武井）。
2,2021/05/13 13:43:53,tetsuya@sat.t.u-tokyo.ac.jp,小林 徹也,0.0,3.0,2.0,0.0,2.0,0.0,0.0,...,東京大学・若本研、理化学研究所・秋山研、慶応大学・舟橋研、近畿大学・山縣研,なし,なし,なし,American Academy of Sleep Medicine (AASM) Youn...,"C班共同会議; 化学走性の会議(非公開, 石島研、小林研、神野圭太 Yale Univers...",なし,なし,なし,なし
11,2021/05/14 14:08:17,amamori@fbs.osaka-u.ac.jp,石島 秋彦,1.0,1.0,NaN,NaN,NaN,NaN,2.0,...,東京大学・竹内研、東京大学・小林研、大阪大学・南野研,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R２年8月から、育休後の蔡助教が分担者として計画班に参加しました。石島研では、二人の保育園児...
10,2021/05/14 9:02:56,k-aoki@nibb.ac.jp,青木 一洋,1.0,2.0,2.0,0.0,0.0,0.0,1.0,...,東京大学・伊藤創祐研、広島大学・本田研,プリンストン大学・Toettcher lab、イエール大学・Emonet lab（神野君と）,NaN,NaN,第72回日本細胞生物学会大会 若手優秀発表賞 後藤祐平 2020年6月,B班内共同会議（2カ月に1回）,NaN,NaN,NaN,2020年3月13日第31回自然科学研究機構シンポジウムで一般向けに研究の話をしました（アウ...
23,2021/05/19 17:20:58,kat@kaztake.org,竹内 一将,0.0,2.0,2.0,0.0,0.0,0.0,1.0,...,領域関係：東京大学・若本研、東京大学・沙川研、理研・川口研（深井さん）、東京大学・田久保さん...,上海航空大・佐野研、ソルボンヌ大学（旧パリ第６大学）・LPTMC研、ESPCI・Gulliv...,NaN,NaN,"・T. Shimaya, EBSA poster prize, Jul.24, 2019\n...",アクティブマター勉強会（岡田研、竹内研、川口研）,竹内一将研究室（C02班、代表）の竹内一将さん（准教授）、⻄口大貴さん（助教、C02班分担）...,NaN,・竹内一将、第64回 物性若手夏の学校 講義「KPZ普遍クラス －厳密解と実験が奏でる非平衡...,海外学生の研究滞在受入（実施1件、今年度予定2件）
24,2021/05/19 18:18:31,sosuke.ito@ubi.s.u-tokyo.ac.jp,伊藤 創祐,NaN,1.0,1.0,0.0,1.0,0.0,0.0,...,京都大学・佐々研、理研・脳科学総合研究センター,AMOLF・P. R. ten Wolde lab,NaN,NaN,芦田慶太(ポスドク)、2020/03 - 藤原奨学基金 藤原賞,エントロピー生成推定勉強会 (沙川研、伊藤研、岡田研）、B班内共同会議、佐々研との最適輸送理...,芦田慶太(ポスドク) 、青木研、2020年3月に約1週間、細胞イメージング実験手法の習得のため,NaN,NaN,伊藤研の学生である吉村耕平さんが、佐々研究室でセミナーを行った。
25,2021/05/19 19:46:01,seto@ap.t.u-tokyo.ac.jp,沙川 貴大,NaN,3.0,4.0,NaN,NaN,NaN,NaN,...,学習院大学、京都大学、東北大、東海大,University of Seoul、Konkuk University,NaN,NaN,日本物理学会 学生優秀2021年3月発表賞（領域11）大坪舜 2021年3月、田中昭二賞（物...,NaN,NaN,NaN,NaN,.
26,2021/05/20 12:02:03,sasa@scphys.kyoto-u.ac.jp,佐々 真一,1.0,1.0,NaN,NaN,1.0,1.0,1.0,...,茨城大学・中川研、慶応大学・斎藤研、東北大学・鳥谷部研、名古屋大学・宮崎研,NaN,NaN,NaN,"Andreas Dechant, Best paper award for ""Multid...",A班内共同セミナー,NaN,NaN,NaN,すでに書いたこと以上のことは思いつきません。
32,2021/05/20 14:44:34,matsuoka@fbs.osaka-u.ac.jp,松岡 里実,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,理研BDR田中陽チーム(令和3年度より契約書あり),Johns Hopkins大学Peter Devreotes Lab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,日本生物物理学会の学会誌副編集長として学会誌でのダイバーシティ特集を企画・実行
33,2021/05/20 14:48:17,cssawai@mail.ecc.u-tokyo.ac.jp,澤井 哲,2.0,0.0,6.0,0.0,2.0,0.0,2.0,...,NaN,Lyon大学 Jean-Paul Rieu,NaN,NaN,NaN,猪股研究室 澤井研究室 ゼブラフィッシュ飼育講習会、2019年9月5日神戸 理化学研究所。C...,NaN,NaN,澤井哲「アリストテレスがあっていて、ニュートンは間違っていた？ 細胞の生物物理学序論」、動画...,科研費(若手研究)獲得、研究室の学生は10名中6名が女性など、若手育成を進めている。


In [25]:
wb = openpyxl.load_workbook("./inputfiles/R3中間評価報告書（1_領域全体）（13参考データExcel版）.xlsx")
ws = wb.worksheets[0]


In [28]:
# 研究代表者人数
## R1 計画
ws['C8'].value=len(allkeikakuPIs)

## R1 公募
ws['D8'].value=0

## R2 計画
ws['E8'].value=len(allkeikakuPIs)

## R2 公募
ws['F8'].value=allgroupnames.count('D')


# 研究分担者人数
## R1 計画
ws['C9'].value=np.sum(keikakudata['c03'])

## R2 計画
ws['E9'].value=np.sum(keikakudata['c09'])

#研究協力者
#若手研究者[1]
#外国人研究者
#ポスドク[2]
#RA等[2]
keikakulist=[col+'{0:02d}'.format(i) for col in ['C','E'] for i in range(10,15) ]
buntanlist=[col+'{0:02d}'.format(i)  for col in ['D','F'] for i in range(10,15)]
datapositions=['c'+'{0:02d}'.format(i) for i in [4,5,6,7,8,10,11,12,13,14]]
for d,kei,bun in zip(datapositions,keikakulist,buntanlist):
    ws[kei].value=keikakudata[d].astype(float).sum()
    ws[bun].value=buntandata[d].astype(float).sum()


In [30]:
allgroupnames

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D']

In [29]:
wb.save(file_name)